In [5]:
import numpy as np
import pandas as pd
from obspy.taup import TauPyModel
from obspy.clients.syngine import Client
from obspy.geodetics import locations2degrees
from obspy import UTCDateTime
from tqdm import tqdm

# Initialize Syngine client and TauPyModel for Mars
client = Client()
MARS_MODEL = "mars_insightkks21gp_2s"
mars_model_path = r"\Geophysical_model1.npz"
mars_model = TauPyModel(model=mars_model_path)

MAX_RETRIES = 3


def calculate_dynamic_shadow_zone(cmb_depth, iocb_depth, ray_param, epi_dist):
    """
    Determine if the event is in the shadow zone based on the ray parameter,
    epicentral distance, and core-mantle/inner-core boundaries.
    """
    # Ray parameters associated with phases interacting with the core
    shadow_phases = ["PKP", "SKS", "PKIKP", "SKIKS"]

    # If ray parameter or epicentral distance indicates interaction with the core
    if epi_dist >= 103.0 and epi_dist <= 142.0:
        return True

    if cmb_depth <= ray_param <= iocb_depth:
        return True

    return False


def generate_mars_synthetic(event_id,
                            source_lat, source_lon,
                            depth_km, magnitude,
                            rec_lat, rec_lon,
                            duration_s=1800.0,
                            cmb_depth=1549.732,
                            iocb_depth=3389.000):
    """
    Generate a single synthetic Mars seismogram with enhanced feature extraction.
    """
    try:
        # Compute epicentral distance
        epi_dist = locations2degrees(source_lat, source_lon, rec_lat, rec_lon)

        # Skip invalid distances
        if epi_dist < 5.0 or epi_dist > 150.0:
            return "distance", epi_dist

        # Calculate travel times using TauPyModel
        arrivals = mars_model.get_travel_times(
            source_depth_in_km=depth_km,
            distance_in_degree=epi_dist,
            phase_list=["P", "S", "PKP", "SKS", "PKIKP", "SKIKS"]
        )

        if not arrivals:
            return "no_travel_times", epi_dist

        # Extract ray parameter and classify shadow zone
        ray_param = arrivals[0].ray_param if arrivals else None
        shadow_zone = calculate_dynamic_shadow_zone(cmb_depth, iocb_depth, ray_param, epi_dist)

        # Attempt waveform retrieval
        retry_count = 0
        st = None
        while retry_count < MAX_RETRIES:
            try:
                st = client.get_waveforms(
                    model=MARS_MODEL,
                    sourcelatitude=source_lat,
                    sourcelongitude=source_lon,
                    sourcedepthinmeters=min(depth_km * 1000, 100000),
                    receiverlatitude=rec_lat,
                    receiverlongitude=rec_lon,
                    components="Z",
                    units="displacement",
                    origintime=UTCDateTime("2025-01-01T00:00:00"),
                    starttime=0.0,
                    endtime=min(duration_s, 3600.0)
                )
                if not st:
                    raise ValueError("No data received.")
                break
            except Exception:
                retry_count += 1
                if retry_count == MAX_RETRIES:
                    return "waveform", None

        # Feature extraction
        amplitude = np.max([abs(tr.data).max() for tr in st])
        all_data = np.concatenate([tr.data for tr in st])
        rms = np.sqrt(np.mean(all_data**2))

        # Dominant frequency
        st_z = st.select(component="Z")
        if st_z:
            trZ = st_z[0]
            freqs = np.fft.rfftfreq(len(trZ.data), d=trZ.stats.delta)
            spec = np.abs(np.fft.rfft(trZ.data))
            dom_idx = np.argmax(spec)
            dom_freq = freqs[dom_idx]
        else:
            dom_freq = None

        duration = st[0].stats.endtime - st[0].stats.starttime

        return {
            "Event ID": event_id,
            "Source Lat": source_lat,
            "Source Lon": source_lon,
            "Receiver Lat": rec_lat,
            "Receiver Lon": rec_lon,
            "Depth (km)": depth_km,
            "Magnitude": magnitude,
            "Epicentral Distance (deg)": epi_dist,
            "Shadow Zone": int(shadow_zone),
            "Ray Parameter": ray_param,
            "Max Amplitude": amplitude,
            "RMS Amplitude": rms,
            "Dominant Frequency (Hz)": dom_freq,
            "Trace Duration (s)": duration
        }
    except Exception as e:
        print(f"Error generating Event ID {event_id}: {e}")
        return None


def create_martian_dataset(num_samples=10,
                           outfile="Synthetic dataset.csv",
                           duration_s=1800.0,
                           cmb_depth=1549.732,
                           iocb_depth=3389.000):
    data_records = []
    shadow_count = 0
    non_shadow_count = 0
    event_id = 1

    print("Generating synthetic dataset with enhanced features...")
    for _ in tqdm(range(num_samples * 2)):
        src_lat = np.random.uniform(-90, 90)
        src_lon = np.random.uniform(-180, 180)
        depth_km = np.random.uniform(0, 100)
        magnitude = np.random.uniform(2.0, 4.5)

        rec_lat = np.random.uniform(-90, 90)
        rec_lon = np.random.uniform(-180, 180)

        result = generate_mars_synthetic(
            event_id=event_id,
            source_lat=src_lat,
            source_lon=src_lon,
            depth_km=depth_km,
            magnitude=magnitude,
            rec_lat=rec_lat,
            rec_lon=rec_lon,
            duration_s=duration_s,
            cmb_depth=cmb_depth,
            iocb_depth=iocb_depth
        )

        if isinstance(result, dict):
            if result["Shadow Zone"] == 1 and shadow_count < num_samples / 2:
                data_records.append(result)
                shadow_count += 1
                event_id += 1
            elif result["Shadow Zone"] == 0 and non_shadow_count < num_samples / 2:
                data_records.append(result)
                non_shadow_count += 1
                event_id += 1

        if shadow_count == non_shadow_count == num_samples / 2:
            break

    print(f"\nBalanced Dataset Summary:")
    print(f"Shadow Events: {shadow_count}")
    print(f"Non-Shadow Events: {non_shadow_count}")
    print(f"Total Events: {len(data_records)}")

    df = pd.DataFrame(data_records)
    df.to_csv(outfile, index=False)
    print(f"Saved {len(df)} events to {outfile}")

    return df


if __name__ == "__main__":
    df = create_martian_dataset(num_samples=1500, outfile="Synthetic dataset.csv")


Generating synthetic dataset with enhanced features...


100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [2:02:48<00:00,  2.46s/it]


Balanced Dataset Summary:
Shadow Events: 725
Non-Shadow Events: 750
Total Events: 1475
Saved 1475 events to mars_data_balanced.csv


In [2]:
df.head()

,Event ID,Source Lat,Source Lon,Receiver Lat,Receiver Lon,Depth (km),Magnitude,Epicentral Distance (deg),Shadow Zone,Ray Parameter,Max Amplitude,RMS Amplitude,Dominant Frequency (Hz),Trace Duration (s)
0,1,59.002059,134.609606,-68.957929,65.400545,52.067781,3.735195,137.255597,1,0.000000,0.000052,0.000006,0.017218,1799.984
1,2,68.724000,-34.110777,17.031185,121.709768,50.727388,4.351187,92.497712,0,196.617853,0.000052,0.000010,0.016107,1799.984
2,3,-54.288173,-154.539450,-47.110274,20.096272,46.378994,3.330608,78.499843,0,240.297568,0.000160,0.000024,0.022772,1799.984
3,4,-54.513599,-154.125747,41.543920,-150.926414,35.531775,3.625546,96.096538,0,0.062500,0.000059,0.000013,0.031103,1799.984
4,5,-33.849889,-66.212464,0.891985,-90.846260,24.696271,4.327723,41.741730,0,400.912067,0.000257,0.000091,0.041100,1799.984
